# **Exchange in 2 dots**

The goal here is to reproduce the results from Matlab code.

# Loading modules and defining main variables

In [1]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

import qudipy as qd
import qudipy.potential as pot
import qudipy.exchange as exch

import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import loadmat    # for loading Matlab files
from scipy.interpolate import interp1d

from tqdm import tqdm
import time
import math


import gc   # for memory cleanup

Specifying the dot parameters

In [2]:
csts = qd.Constants('Si/SiO2')

nItinOrbs = [8,2]#[10,12,2,4,]
n_dot_seps = 10

dot_seps = np.linspace(7.5,15, n_dot_seps) * 1E-9;
a = 6.0E-9;     # dot_width
dx_dy = 0.5;

omega = csts.hbar / (csts.me * a ** 2 )
print('omega = {:e} Hz'.format(omega))

omega = 1.683648e+13 Hz


Creating an array of grid parameters objects (for future simulations)

In [3]:
ngridx = 200
ngridy = 200

xgrids = np.linspace( - dot_seps - 5 * a, dot_seps + 5 * a, ngridx).T
ygrids = np.tile(np.linspace( - 5 * a, 5 * a, ngridy), (n_dot_seps, 1))

all_gparams = (pot.GridParameters(xgrids[ idx], ygrids[ idx])
                    for idx in range(n_dot_seps))

# numbers of harmonic orbitals
n_xy_ho = (15,15)
xgrids.shape

(10, 200)

Defining a function that creates a quartic potential

In [4]:
def create_quartic(gparams, d):
    return 1 / 2 * csts.me * omega ** 2 * (
                    1 / ( 4 * d **2) * (gparams.x_mesh ** 2 - d ** 2) ** 2
                        + gparams.y_mesh ** 2
    )

Iterate over dot separations and numbers of single-electron orbitals to calculate the exchange values

In [5]:
#%%script false

exchanges = []

omega_guess = 0.025 * csts.hbar / (csts.me * csts.a_B**2)

for n_se_orbs in nItinOrbs:
    gparams = next(all_gparams)
    exs = np.zeros((n_dot_seps,))
    for idx in tqdm(range(n_dot_seps)):
        gparams.update_potential(create_quartic(gparams, dot_seps[idx]))
        ens, __ = qd.qutils.solvers.solve_many_elec_SE(gparams, n_elec=2, 
                      n_xy_ho=n_xy_ho, 
                      n_se=n_se_orbs, 
                      n_sols=2,
                      consts=csts,
                      optimize_omega=True,
                      omega=omega_guess,
                      opt_omega_n_se=n_se_orbs,
                      cme_dir='..\\tutorials\\QuDiPy tutorial data\\CMEs',
                      spin_subspace=[1]);
        exs[idx] = ens[1] - ens[0]
        print('exchange value is: ', exs[idx] ,
        '\nenergies are: ', ens)
    exchanges.append(exs)


  0%|          | 0/10 [00:00<?, ?it/s]

Begining many body energy calculation...

Optimizing choice of omega in approximating the single electron orbitals...

Optimization terminated successfully.
Found an optimal omega of 2.34E+13.

Done!
Elapsed time is 67.22774648666382 seconds.

Finding 2D harmonic orbitals at origin...

Done!

Finding A matrix...

Done!
Elapsed time is 33.97121548652649 seconds.

Loading harmonic orbital CME matrix from the specified directory

Transforming the CME library to single electron basis...

Done!
Elapsed time is 29.943827629089355 seconds.

Building 2nd quantization Hamiltonian and diagonalizing...
1. The nondiagonal part is built
2. The diagonal part is built
Done!

Elapsed time is 1071.7186801433563 seconds.



 10%|█         | 1/10 [20:35<3:05:20, 1235.57s/it]

Total calculation time: 1234.27 sec, 20.57 min.
Time optimizing omega: 67.23 sec, 5.45% of total.
Time finding A matrix: 33.97 sec, 2.75% of total.
Time transforming CMEs: 29.94 sec, 2.43% of total.
Time building 2nd quantization H: 1071.72 sec, 86.83% of total.
exchange value is:  8.480918795523421e-22 
energies are:  [2.76963197e-21 3.61772385e-21]
Begining many body energy calculation...

Optimizing choice of omega in approximating the single electron orbitals...

Optimization terminated successfully.
Found an optimal omega of 2.70E+13.

Done!
Elapsed time is 47.500558614730835 seconds.

Finding 2D harmonic orbitals at origin...

Done!

Finding A matrix...

Done!
Elapsed time is 34.07759141921997 seconds.

Loading harmonic orbital CME matrix from the specified directory

Transforming the CME library to single electron basis...

Done!
Elapsed time is 30.287296772003174 seconds.

Building 2nd quantization Hamiltonian and diagonalizing...


 10%|█         | 1/10 [40:25<6:03:53, 2425.99s/it]


KeyboardInterrupt: 

# Loading and plotting the exchange data from Matlab


In [ ]:

mat_contents = loadmat('N=2_4_6_8_10_M=15')
mat_contents.keys()


In [ ]:
mat_exchanges = mat_contents['exchange']
mat_dot_seps = np.linspace(7.5,15,100) * 2
mat_exchanges.T[:,:10]


Adding the exchange values from Python code


In [ ]:
exchange_values = np.array([5.729665778336784e-21, 4.959013855226583e-21,   4.476435249829594e-21, 2.2186468216525865e-19 ])

interpolated_exchanges = interp1d(mat_dot_seps/2, mat_exchanges[:,4])

interpolated_exchanges(np.linspace(7.5,15,10)[:4]) / exchange_values


In [ ]:

%matplotlib widget
legends = ('N=2','N=4','N=6','N=8','N=10', 'N=8 data')

plt.figure(figsize=(6,5))
for ex in mat_exchanges.T:
    plt.plot(mat_dot_seps, ex * 42.7E3)
#plt.legend(plot, legends)

plt.plot(np.linspace(15,30,10)[:4], exchange_values*1.6e25, 'r.' )
plt.xscale('linear')
plt.yscale('log')
plt.legend(legends, loc='best', fancybox=True, ncol=2)
plt.xlim((15,30))

plt.show()
